In [ ]:
import albumentations as A
from PIL import Image
import numpy as np
from albumentations.pytorch import ToTensorV2
from albumentations.core.transforms_interface import ImageOnlyTransform

In [ ]:
!wget https://sochi-hub.obs.ru-moscow-1.hc.sbercloud.ru:443/wildlife_project/whale_leus/Whale%20ReId%202.7z

--2025-04-14 16:13:00--  https://sochi-hub.obs.ru-moscow-1.hc.sbercloud.ru/wildlife_project/whale_leus/Whale%20ReId%202.7z
Resolving sochi-hub.obs.ru-moscow-1.hc.sbercloud.ru (sochi-hub.obs.ru-moscow-1.hc.sbercloud.ru)... 46.243.206.34, 46.243.206.35
Connecting to sochi-hub.obs.ru-moscow-1.hc.sbercloud.ru (sochi-hub.obs.ru-moscow-1.hc.sbercloud.ru)|46.243.206.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7624464497 (7.1G) [application/x-7z-compressed]
Saving to: ‘Whale ReId 2.7z’

Whale ReId 2.7z     100%[===================>]   7.10G  19.0MB/s    in 5m 23s  

2025-04-14 16:18:25 (22.5 MB/s) - ‘Whale ReId 2.7z’ saved [7624464497/7624464497]



In [ ]:
!pip install py7zr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 22.7 MB/s eta 0:00:00


In [ ]:
import py7zr
import os

with py7zr.SevenZipFile('/content/Whale ReId 2.7z', mode='r') as archive:
    all_files = archive.getnames()

In [ ]:
archive_path = "/content/Whale ReId 2.7z"
extract_dir = "/content/Whales"

os.makedirs(extract_dir, exist_ok=True)

with py7zr.SevenZipFile(archive_path, mode='r') as archive:
    archive.extractall(path=extract_dir)

In [ ]:
def to_mask(mask_path):
  return Image.open(mask_path)


def apply_mask(img, mask_img):
  image = Image.open(img)
  mask = Image.open(mask_img)

  return Image.composite(image, Image.new("RGB", image.size, (0, 0, 0)), mask)

In [ ]:
apply_mask('/content/whales_processed/train/20_01_frames/crop1_DJI_0002/crop1_DJI_0002_1.jpg',
           '/content/whales_processed/train/20_01_frames/crop1_DJI_0002/crop1_DJI_0002_1.png')

FileNotFoundError: [Errno 2] No such file or directory: '/content/whales_processed/train/20_01_frames/crop1_DJI_0002/crop1_DJI_0002_1.jpg'

In [ ]:
!rm -rf whales_processed

In [ ]:
import shutil
from tqdm import tqdm
import random

def split_whale_data(source_dir, train_ratio=0.8):
    whales = os.listdir(source_dir)
    for whale in tqdm(whales):
        whale_path = os.path.join(source_dir, whale)
        dates = os.listdir(whale_path)
        random.shuffle(dates)

        total_photos = sum(
            len(os.listdir(os.path.join(whale_path, date)))
            for date in dates
        )
        target_test_photos = int(total_photos * (1 - train_ratio))

        train_dates, test_dates = [], []
        test_photo_count = 0

        for date in dates:
            date_photos = len(os.listdir(os.path.join(whale_path, date)))
            if test_photo_count + date_photos <= target_test_photos:
                test_dates.append(date)
                test_photo_count += date_photos
            else:
                train_dates.append(date)

        if not test_dates:
            test_dates = [random.choice(dates)]
            train_dates = [d for d in dates if d not in test_dates]

        for date in train_dates:
            src = os.path.join(whale_path, date)
            dst = os.path.join("whales_processed/train", whale, date)
            os.makedirs(dst, exist_ok=True)
            src_imgs = os.listdir(src)
            for src_img in src_imgs:
                if src_img.split('.')[1] == 'jpg':
                    stem = src_img.split('.')[0]
                    original_path = os.path.join(src, src_img)
                    mask_path = os.path.join(src, stem + ".png")
                    apply_mask(original_path, mask_path).save(os.path.join(dst, stem + ".jpg"))

        for date in test_dates:
            src = os.path.join(whale_path, date)
            dst = os.path.join("whales_processed/test", whale, date)
            os.makedirs(dst, exist_ok=True)
            src_imgs = os.listdir(src)
            for src_img in src_imgs:
                if src_img.split('.')[1] == 'jpg':
                    stem = src_img.split('.')[0]
                    original_path = os.path.join(src, src_img)
                    mask_path = os.path.join(src, stem + ".png")
                    apply_mask(original_path, mask_path).save(os.path.join(dst, stem + ".jpg"))

split_whale_data("/content/Whales/Whale ReId 2")

100%|██████████| 102/102 [20:14<00:00, 11.91s/it]


In [ ]:
def flatten_directory_structure(root_dir):
    for split in ["train", "test"]:
        split_path = os.path.join(root_dir, split)
        if not os.path.exists(split_path):
            continue

        for whale in tqdm(os.listdir(split_path), desc=f"Processing {split}"):
            whale_path = os.path.join(split_path, whale)

            for root, dirs, files in os.walk(whale_path):
                for file in files:
                    if file.lower().endswith(('.jpg')):
                        src_path = os.path.join(root, file)

                        date_folder = os.path.basename(root)
                        new_filename = f"{date_folder}_{file}"

                        dest_path = os.path.join(whale_path, new_filename)

                        shutil.move(src_path, dest_path)

            for root, dirs, files in os.walk(whale_path, topdown=False):
                for dir in dirs:
                    dir_path = os.path.join(root, dir)
                    if not os.listdir(dir_path):
                        os.rmdir(dir_path)

flatten_directory_structure("whales_processed")

Processing test: 100%|██████████| 102/102 [00:00<00:00, 190.96it/s]


In [ ]:
!tar -cvf masked_dataset.tar /content/whales_processed

Выходные данные были обрезаны до нескольких последних строк (5000).
/content/whales_processed/test/21_85_frames/crop_DJI_0042_crop_DJI_0042_652.jpg
/content/whales_processed/test/21_85_frames/crop_DJI_0042_crop_DJI_0042_90.jpg
/content/whales_processed/test/21_85_frames/crop_DJI_0042_crop_DJI_0042_435.jpg
/content/whales_processed/test/21_85_frames/crop_DJI_0042_crop_DJI_0042_443.jpg
/content/whales_processed/test/21_85_frames/crop_DJI_0042_crop_DJI_0042_270.jpg
/content/whales_processed/test/21_85_frames/crop_DJI_0042_crop_DJI_0042_218.jpg
/content/whales_processed/test/21_85_frames/crop_DJI_0042_crop_DJI_0042_570.jpg
/content/whales_processed/test/21_85_frames/crop_DJI_0042_crop_DJI_0042_750.jpg
/content/whales_processed/test/21_85_frames/crop_DJI_0042_crop_DJI_0042_390.jpg
/content/whales_processed/test/21_85_frames/crop_DJI_0042_crop_DJI_0042_158.jpg
/content/whales_processed/test/21_85_frames/crop_DJI_0042_crop_DJI_0042_772.jpg
/content/whales_processed/test/21_85_frames/crop_DJI_